# Test2-Testing Set : Dataset-taiex -o/w sentiment

In [4]:
from brbmdl import brbm, dataset_loading, process_dataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pickle

In [5]:
features = ['macd', 'rsi', 'k', 'd', 'r', 'close',
            'before1day', 'before2day', 'before3day', 'before4day','before5day']
target = 'dailyreturn'

In [6]:
X_train, y_train = dataset_loading(features, target, './data/taiex_train_data.csv')
X_test, y_test = dataset_loading(features, target, './data/taiex_test_data.csv')

y_train_label = process_dataset(y_train, target)
y_test_label = process_dataset(y_test, target)

In [ ]:
brb_accs, brb_pres, brb_recs, brb_f1s = [], [], [], []
N = 1
for i in range(N):
    brb_model = brbm(features, target)
    brb_model.set_ante_labels(['superlow', 'low', 'medium', 'high', 'superhigh'])
    brb_model.set_cons_labels(['down', 'up'])
    # brb_model.set_all_ante_fuzzy_ranges(X_train, y_train_label, features)
    brb_model.set_all_ante_fuzzy_ranges('./dtree_models/decision_tree_model2.pkl', features)
    cons_fuzzy_range = [
        [-1.0, -0.7, -0.25, 0.0],
        [0.0, 0.25, 0.7, 1.0]
    ]
    brb_model.set_cons_fuzzy_range(cons_fuzzy_range)

    brb_model.inital_database(X_train, y_train)
    brb_model.inital_rulebase()
    brb_model.preprocess_database()
    brb_model.optimize(X_train, y_train, features, target)

    y_pred = brb_model.predict(X_test)
    accuracy = accuracy_score(y_test_label, y_pred)
    precision = precision_score(y_test_label, y_pred, average='macro')
    recall = recall_score(y_test_label, y_pred, average='macro')
    f1 = f1_score(y_test_label, y_pred, average='macro')

    brb_accs.append(accuracy)
    brb_pres.append(precision)
    brb_recs.append(recall)
    brb_f1s.append(f1)

print(f'BRBMDL Accuracy: {np.mean(brb_accs) * 100:.2f}')
print(f'BRBMDL Precision: {np.mean(brb_pres) * 100:.2f}')
print(f'BRBMDL Recall: {np.mean(brb_recs) * 100:.2f}')
print(f'BRBMDL F1 Score: {np.mean(brb_f1s) * 100:.2f}')


Optimize: 100%|██████████| 1686/1686 [04:32<00:00,  6.20it/s]


BRBMDL Accuracy: 80.77
BRBMDL Precision: 80.92
BRBMDL Recall: 81.25
BRBMDL F1 Score: 80.74


In [ ]:
# with open('./dtree_models/decision_tree_model2.pkl', 'wb') as f:
#     pickle.dump(brb_model.dtree, f)

In [ ]:
# with open('./dtree_models/decision_tree_model2.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)